## Experiment 01
### Richardson et. al (2002) Experiment 01 with Language Models instead of Humans:

The subjects were presented with a single page,
containing a list of the verbs and four pictures, labelled A to
D. Each one contained a circle and a square aligned along a
vertical or horizontal axis, connected by an arrow pointing
up, down, left or right. Since we didn't expect any
interesting item variation between left or right placement of
the circle or square, the horizontal schemas differed only in
the direction of the arrow.
For each sentence, subjects were asked to select one of
the four sparse images that best depicted the event described
by the sentence (Figure 1)
The items were randomised in three different orders, and
crossed with two different orderings of the images. The six
lists were then distributed randomly to subjects.

### Step 01: Creating a prompt that is as close to the paper as possible

In [106]:
import random
random.seed(1337)


def convert_to_float(value):
    try:
        return float(value)
    except ValueError:
        return value

with open("../../data/richardson_actions.txt", "r") as d_in:
    lines = [line.split() for line in d_in.readlines()]

output = []
for entry in lines:
    new_entry = [convert_to_float(item) for item in entry]
    
    if isinstance(new_entry[1],str):
        new_entry[0] = " ".join(new_entry[:2])
        del new_entry[1]
    output.append(new_entry)

richardson_data = dict()
for elem in output:
    richardson_data[elem[0]] = [i for i in elem[1:]]

richardson_data[0] = ["C:◯↑▢", "D:◯↓▢","B:◯←▢ ","A:◯→▢"]

# Randomizing Richardson's data
action_words = list(richardson_data.keys())
random.shuffle(action_words)

#count=1
#for line in action_words:
#    if line != 0:
#        print(str(count)+". ◯ "+line+" ▢ \\")
#        count+=1

In [2]:
paper_prompt = "You are asked to select one of the four images that best depicts the event described by the following sentences. \
Image A: \
◯→▢ \
\
Image B: \
◯←▢ \
 \
Image C: \
◯ \
↑ \
▢ \
\
Image D: \
◯ \
↓ \
▢ \
\
Sentences: \
1. ◯ hunted ▢ \
2. ◯ showed ▢ \
3. ◯ succeeded ▢ \
4. ◯ pointed at ▢ \
5. ◯ pushed ▢ \
6. ◯ hoped ▢ \
7. ◯ walked ▢ \
8. ◯ gave to ▢ \
9. ◯ respected ▢ \
10. ◯ smashed ▢ \
11. ◯ argued with ▢ \
12. ◯ sank ▢ \
13. ◯ rested ▢ \
14. ◯ offended ▢ \
15. ◯ pulled ▢ \
16. ◯ perched ▢ \
17. ◯ regretted ▢ \
18. ◯ bombed ▢ \
19. ◯ obeyed ▢ \
20. ◯ lifted ▢ \
21. ◯ flew ▢ \
22. ◯ impacted ▢ \
23. ◯ wanted ▢ \
24. ◯ increased ▢ \
25. ◯ warned ▢ \
26. ◯ floated ▢ \
27. ◯ tempted ▢ \
28. ◯ rushed ▢ \
29. ◯ owned ▢ \
\
For sentence 1) choosing from [A,B,C,D] best image is "


### Step 02: Test model with this `paper prompt`

#### Function to evaluate against Richardson's findings

In [3]:
import torch
from transformers import GPT2Tokenizer, OPTForCausalLM, AutoTokenizer, AutoModelForCausalLM
from transformers import GPTNeoXForCausalLM, GPTNeoXTokenizerFast, GPTNeoForCausalLM, GPT2Tokenizer
import torch.nn.functional as F
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm
import json
from scipy import stats
from collections import Counter
import subprocess

with open("../../hf.key", "r") as f_in:
    hf_key = f_in.readline().strip()
subprocess.run(["huggingface-cli", "login", "--token", hf_key])

server_model_path = "/mounts/data/corp/huggingface/"
model_type = "meta-llama"
model_name = "Llama-2-70b-hf"

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /mounts/data/corp/huggingface/token
Login successful


In [4]:
#tokenizer = GPTNeoXTokenizerFast.from_pretrained(model_prefix+"-"+model_size, device_map="auto")
#model = GPTNeoXForCausalLM.from_pretrained(model_prefix+"-"+model_size, device_map="auto")

#tokenizer = GPT2Tokenizer.from_pretrained(model_prefix+"-"+model_size, device_map="auto")
#model = OPTForCausalLM.from_pretrained(model_prefix+"-"+model_size, device_map="auto")

#tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-13b-hf", use_auth_token=True, device_map="auto")
#model = AutoModelForCausalLM.from_pretrained(server_model_path+"llama/llama-13b", use_auth_token=True, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_type+"/"+model_name, use_auth_token=True, device_map="auto")
model = AutoModelForCausalLM.from_pretrained(server_model_path+model_type+"/"+model_name, use_auth_token=True, device_map="auto")

/mounts/work/pwicke/miniconda3/envs/schemas/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/mounts/work/pwicke/miniconda3/envs/schemas/lib/python3.8/site-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [6]:
# Step 1: Tokenize the prompt
input_ids = tokenizer.encode(paper_prompt, return_tensors="pt")

# Step 2: Generate the model input
max_length = input_ids.size(1)  + 20 # Adjust '20' as needed to control the maximum length of the generated answer.
output = model.generate(input_ids, max_length=max_length, num_return_sequences=1)

# Step 3: Decode the generated output to get the answer
generated_answer = tokenizer.decode(output[0], skip_special_tokens=True)


In [7]:
print("Generated answer:\n\n", generated_answer[len(paper_prompt):])

Generated answer:

 ◯→▢. For sentence 2) choosing from [A,B,


### Step 03: Prompting that is maximally friendly for the LLM

In [31]:
for action_word in action_words[:10]:
    
    print("ACTION: "+action_word)

    friendly_prompt = "Select the image that best represents the event described by the sentence: "+action_word+"\n[◯→▢]\n\n[◯←▢]\n\n[◯\n↑\n▢]\n\n[◯\n↓\n▢]\n\nThe best representation is [◯"
    
    input_ids = tokenizer.encode(friendly_prompt, return_tensors="pt")
    max_length = input_ids.size(1)  + 10
    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1)
    generated_answer = tokenizer.decode(output[0], skip_special_tokens=True)  
    print("Generated answer:\n\n", generated_answer[len(friendly_prompt):])

prompt_B = "Select the image that best represents the event described by the sentence: "+action_word+"\n[◯→▢]\n\n[◯←▢]\n\n[◯\n↑\n▢]\n\n[◯\n↓\n▢]\n\nThe best representation is [◯"
#prompt_C = "Choose the best image for the word:\nUP: ↑ \nDOWN: ↓ \nLEFT: → \nRIGHT: ← \n"+action_word.upper()+": "
#prompt_D = "Choosing from UP, DOWN, LEFT and RIGHT, the word \'"+action_word.upper()+"\' is best respresented by the word "
#prompt_B = "Of these four: A: ◯→▢ B: ◯←▢ C: ◯ ↑ ▢ D: ◯ ↓ ▢ Which one best describes \"◯ "+action_word+" ▢\" ?"

ACTION: hunted


/mounts/work/pwicke/miniconda3/envs/schemas/lib/python3.8/site-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Generated answer:

 →▢].

### 
ACTION: showed
Generated answer:

 ←▢].

### 
ACTION: succeeded
Generated answer:

 →▢].

### 
ACTION: pointed at
Generated answer:

 ←▢].

Select the image
ACTION: pushed
Generated answer:

 ←▢].

Select the image
ACTION: hoped
Generated answer:

 
↓
▢].

Select
ACTION: walked
Generated answer:

 →▢].

### 
ACTION: gave to
Generated answer:

 ←▢].

Select the image
ACTION: respected
Generated answer:

 
↓
▢].

Select
ACTION: smashed
Generated answer:

 →▢].

Select the image


### Step 03: Test pipline with logprobs and `friendly prompt`

In [9]:
from sidemethods import logprobs_from_prompt, proc, proc_lower, prob_of_ending, calculate_accuracy, calculate_accuracies, store_accuracies

In [10]:
def get_action_prompt(action):
    return "Select the image that best represents the event described by the sentence: "+action+"\n[◯→▢]\n\n[◯←▢]\n\n[◯\n↑\n▢]\n\n[◯\n↓\n▢]\n\nThe best representation is [◯"
    
def get_test_action(country):
    return country+"'s captitol is "

In [16]:
richardson_data

{'fled': [7.2, 4.2, 80.8, 7.8],
 'pointed at': [7.2, 3.6, 0.0, 89.2],
 'pulled': [6.0, 5.4, 75.4, 13.2],
 'pushed': [7.2, 3.6, 1.2, 8.0, 8.0],
 'walked': [9.0, 3.6, 2.0, 4.0, 62.9],
 'hunted': [9.6, 20.4, 1.8, 68.3],
 'impacted': [7.2, 37.1, 3.0, 52.7],
 'perched': [1.0, 2.0, 7.0, 6.0, 6.6, 5.4],
 'showed': [1.0, 5.0, 9.0, 10.2, 65.9],
 'smashed': [3.6, 66.5, 1.2, 28.7],
 'bombed': [4.8, 86.8, 1.8, 6.6],
 'flew': [37.7, 44.3, 15.0, 3.0],
 'floated': [32.9, 56.3, 7.8, 3.0],
 'lifted': [87.4, 9.6, 2.4, 0.6],
 'sank': [22.2, 71.9, 4.2, 1.8],
 'argued with': [11.4, 13.8, 12.6, 62.3],
 'gave to': [8.4, 9.6, 1.2, 80.8],
 'offended': [9.0, 31.7, 24.6, 34.7],
 'rushed': [10.2, 10.8, 23.4, 55.1],
 'warned': [10.8, 22.2, 6.0, 61.1],
 'owned': [5.4, 55.7, 18.6, 20.4],
 'regretted': [19.8, 24.0, 41.3, 1.0, 5.0],
 'rested': [14.4, 36.5, 40.1, 9.0],
 'tempted': [16.8, 11.4, 45.5, 26.3],
 'wanted': [15.6, 7.8, 15.6, 61.1],
 'hoped': [45.5, 15.6, 7.2, 31.7],
 'increased': [73.7, 7.2, 9.6, 9.0],
 'obey

In [39]:
friendly_prompt_choices = dict()

#action_words= ["Germany", "Australia", "Japan", "Israel", "France"]

for action_word in tqdm(action_words):
    #print(action_word)
    if action_word == 0:
        continue
    
    start = get_action_prompt(action_word)
    #start = get_test_action(action_word)
    
    
    answers = {0:"→", 1:"←", 2:"↑", 3:"↓"}
    #answers = {0: "Berlin", 1:"Paris", 2:"Canberra", 3:"Tokyo"}

    #start = "nazis are known to be on the political "
    #answers = {0:"UP", 1:"DOWN", 2:"LEFT", 3:"RIGHT"}

    res_ends = []
    for j, end in answers.items():
        input_prompt = proc(start) + ' ' + proc(end)
        logprobs = logprobs_from_prompt(input_prompt, tokenizer, model)
        res = {"tokens": [x for x,y in logprobs],"token_logprobs": [y for x,y in logprobs]}
        res_ends.append(res)

    choosen_answer = (-9999, "")
    
    choice_distribution = []
    for i, answer in answers.items():
        choice_val = prob_of_ending(res_ends[i]['token_logprobs'], res_ends[i]['tokens'])
        if choice_val > choosen_answer[0]:
            choosen_answer = choice_val, answer
        choice_distribution.append(choice_val)
    
    # Richardson has table columns and image sequences mixed up, we align it here
    friendly_prompt_choices[action_word] = []
    friendly_prompt_choices[action_word].append(choice_distribution[2])
    friendly_prompt_choices[action_word].append(choice_distribution[3])
    friendly_prompt_choices[action_word].append(choice_distribution[1])
    friendly_prompt_choices[action_word].append(choice_distribution[0])

    #print("Choice: ", choosen_answer[1])
    #print()




100%|██████████| 30/30 [01:46<00:00,  3.55s/it]


In [40]:
for k, v in friendly_prompt_choices.items():
    print(k, [round(x,2) for x in v])

hunted [-0.78, -1.3, -1.65, -2.21]
showed [-0.79, -1.39, -1.67, -2.22]
succeeded [-0.82, -1.43, -1.72, -2.19]
pointed at [-0.79, -1.42, -1.71, -2.19]
pushed [-0.83, -1.42, -1.63, -2.07]
hoped [-0.8, -1.38, -1.75, -2.29]
walked [-0.78, -1.41, -1.64, -2.11]
gave to [-0.83, -1.43, -1.66, -2.17]
respected [-0.78, -1.41, -1.71, -2.28]
smashed [-0.81, -1.4, -1.7, -2.18]
argued with [-0.78, -1.31, -1.64, -2.33]
sank [-0.81, -1.23, -1.76, -2.28]
rested [-0.77, -1.37, -1.8, -2.25]
offended [-0.79, -1.33, -1.75, -2.38]
pulled [-0.8, -1.4, -1.65, -2.17]
perched [-0.72, -1.38, -1.78, -2.32]
regretted [-0.76, -1.35, -1.71, -2.33]
bombed [-0.73, -1.35, -1.63, -2.18]
obeyed [-0.79, -1.31, -1.69, -2.21]
lifted [-0.76, -1.42, -1.79, -2.21]
flew [-0.74, -1.54, -1.81, -2.14]
impacted [-0.79, -1.32, -1.66, -2.03]
wanted [-0.81, -1.41, -1.67, -2.15]
increased [-0.83, -1.46, -1.93, -2.16]
warned [-0.79, -1.43, -1.67, -2.24]
floated [-0.76, -1.41, -1.8, -2.2]
tempted [-0.78, -1.37, -1.75, -2.24]
rushed [-0.7

No matter how the friendly prompt is adjusted, the weight of the symbols  → is so strong that it will always heavily favor one symbol over all other symbols. It is not feasible to ask for the logprobs in this experiment. 

### Step 04: Suitable `friendly prompt` for zero-shot evaluation


In [110]:
model_choices = dict()
arrows = ['↑', '↓', '←', '→']

for action_word in tqdm(action_words):

    if action_word == 0:
        continue

    friendly_prompt = "Select the image that best represents the event described by the sentence: "+action_word+"\n[◯→▢]\n\n[◯←▢]\n\n[◯\n↑\n▢]\n\n[◯\n↓\n▢]\n\nThe best representation is [◯"
    
    input_ids = tokenizer.encode(friendly_prompt, return_tensors="pt")
    max_length = input_ids.size(1)  + 20
    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1)
    generated_answer = tokenizer.decode(output[0], skip_special_tokens=True)  
    
    model_choices[action_word] = []
    for idx, arrow in enumerate(arrows):
        if arrow in generated_answer[len(friendly_prompt):]:
            model_choices[action_word].append(1)
        else:
            model_choices[action_word].append(0)


  0%|          | 0/31 [00:00<?, ?it/s]/mounts/work/pwicke/miniconda3/envs/schemas/lib/python3.8/site-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
100%|██████████| 31/31 [04:15<00:00,  8.23s/it]


In [132]:
print("Action", "C:◯↑▢", 'D:◯↓▢', 'B:◯←▢ ', 'A:◯→▢')
for k,v in model_choices.items():
    print(k,v)


with open("results/exp01a_"+model_name+".txt", "w") as f_out:
    f_out.write("Action\tC:◯↑▢\tD:◯↓▢\tB:◯←▢\tA:◯→▢\n")
    for k,v in model_choices.items():
        f_out.write(k+"\t"+"\t".join([str(x) for x in v])+"\n")


    

Action C:◯↑▢ D:◯↓▢ B:◯←▢  A:◯→▢
walked [0, 0, 0, 1]
perched [0, 1, 0, 0]
respected [0, 1, 0, 0]
fled [0, 0, 1, 0]
pulled [0, 0, 1, 0]
wanted [0, 0, 0, 1]
pushed [0, 0, 1, 0]
argued with [0, 0, 1, 0]
obeyed [0, 0, 1, 0]
showed [0, 0, 1, 0]
sank [0, 1, 0, 0]
lifted [0, 0, 1, 0]
regretted [0, 0, 1, 0]
gave to [0, 0, 1, 0]
pointed at [0, 0, 1, 0]
impacted [0, 0, 1, 0]
owned [0, 0, 1, 0]
smashed [0, 0, 0, 1]
increased [0, 0, 1, 0]
floated [0, 0, 0, 1]
bombed [0, 0, 0, 1]
hunted [0, 0, 0, 1]
tempted [0, 0, 0, 1]
hoped [0, 1, 0, 0]
rushed [0, 0, 0, 1]
flew [0, 0, 0, 1]
rested [0, 0, 1, 0]
offended [0, 0, 1, 0]
succeeded [0, 0, 0, 1]
warned [0, 0, 1, 0]


### Step 05: Compare with Richardson's data

In [145]:
# binarize richardson's data
with open("results/exp01_richardson_categorial.txt", "w") as f_out:
    f_out.write("Action\tC:◯↑▢\tD:◯↓▢\tB:◯←▢\tA:◯→▢\n")
    for k, v in richardson_data.items():
        vals = [0,0,0,0]
        vals[v.index(max(v))] = 1
        f_out.write(k+"\t"+"\t".join([str(x) for x in vals])+"\n")